In [11]:
import pandas as pd
import regionmask
import xarray as xr

In [12]:
def process_ds(path):
    ds = xr.open_zarr(path, consolidated=True)

    ds['ELUC'] = ds['ELUC'].shift(time=1)
    ds['ELUC_diff'] = ds['ELUC_diff'].shift(time=1)
    ds['time'] = ds.time - 1

    ds = ds.stack(latlon=('lat', 'lon'))

    mask = ds['ELUC_diff'].isnull().compute()
    ds = ds.where(~mask, drop=True)

    country_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask(ds)
    ds = ds.assign_coords(country=country_mask)
    
    return ds

In [13]:
def convert_to_dataframe(da):
    df = da.to_dataframe()
    df.index = df.index.set_names(['time', 'i_lat', 'i_lon'])
    df = df.reset_index()
    return df

In [19]:
PATH_TO_DATASET = "processed/merged_aggregated_dataset_1850_2022.zarr.zip"
ds = process_ds(PATH_TO_DATASET)

countries = regionmask.defined_regions.natural_earth_v5_0_0.countries_110
countries_df = countries.to_dataframe()

# Change this
country_list = ["GB", "BR", "CH"]
country_id_list = countries_df.index[countries_df['abbrevs'].isin(country_list)].tolist()

da = ds.where(ds.country.isin(country_id_list), drop=True)
df = convert_to_dataframe(da)
path = ""
for c in country_list:
    path += c + "_"
path += "eluc.csv"
df.to_csv(f"processed/path.lower()")